In [1]:
#Import required libraries

import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
#Set constants for training
seed = 543
log_interval = 10
gamma = 0.99

env = gym.make('CartPole-v1')
env.reset()
state_shape = env.observation_space.shape[0]
no_of_actions = env.action_space.n
print(state_shape)
print(no_of_actions)
torch.manual_seed(seed)


4
2


In [3]:
class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self,state_shape = 4,action_size = 2 ):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(state_shape, 128)
        # actor's layer
        self.action_head = nn.Linear(128, action_size)
        # action & reward buffer
        self.saved_actions = []
        self.episode_rewards = []
        self.episode_states = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.affine1(x))
        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)
        return action_prob


In [4]:
model = Policy(state_shape=state_shape,action_size=no_of_actions)
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()
max_len = 10000 # maximum number of iteration for episode to end

In [5]:

def select_action(state):
    state = torch.from_numpy(state).float()
    probs = model(state)
    #print(probs)
    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)
    action = m.sample()
    model.saved_actions.append(m.log_prob(action))
    # the action to take (left or right)
    return action.item()


def finish_episode(Returns,state_number):
    policy_losses = [] # list to save actor (policy) loss
    # calculate the true value using rewards returned from the environment  
    R = torch.tensor(Returns)
    log_prob = model.saved_actions[state_number]
    policy_losses.append(-torch.mul(R,log_prob))
    optimizer.zero_grad()
    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum()
    #print(loss)
    torch.autograd.set_detect_anomaly(True)
    loss.backward()
    optimizer.step()
    


def train():
    running_reward = 10
    # run infinitely many episodes
    for i_episode in range(2000):
        # reset environment and episode reward
        state = env.reset()
        ep_reward = 0
        # for each episode, only run 9999 steps so that we don't
        # infinite loop while learning
        for t in range(1, max_len):
            # select action from policy
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            model.episode_states.append(state)
            model.episode_rewards.append(reward)
            ep_reward += reward
            if done:
                break
            
        for i in range(len(model.saved_actions)):
            R = 0
            print(len(model.saved_actions))
            for r in model.episode_rewards[i::-1]:
                # calculate the discounted value
                R = r + gamma * R
            print(i)  
            finish_episode(Returns=R, state_number= i)
            
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

        print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))
        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

In [6]:
train()

10
0


/home/navin/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/navin/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:266: UserWarning: Error detected in AddmmBackward0. No forward pass information available. Enable detect anomaly during forward pass for more information. (Triggered internally at ../torch/csrc/autograd/python_anomaly_mode.cpp:91.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


10
1


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [128, 2]], which is output 0 of AsStridedBackward0, is at version 2; expected version 1 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!